# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edward

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/Alibaba-NLP/Tongyi-DeepResearch-30B-A3B',
 '/tencent/SRPO',
 '/openbmb/VoxCPM-0.5B',
 '/ibm-granite/granite-docling-258M',
 '/google/vaultgemma-1b',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Wan-AI/Wan2.2-Animate',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/abdul9999/NoWatermark',
 '/spaces/IndexTeam/IndexTTS-2-Demo',
 '/spaces',
 '/datasets/HuggingFaceFW/finepdfs',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/InternRobotics/OmniWorld',
 '/datasets/LucasFang/FLUX-Reason-6M',
 '/datasets/HuggingFaceM4/FineVision',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transforme

In [12]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'discuss page', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Alibaba-NLP/Tongyi-DeepResearch-30B-A3B
Updated
5 days ago
•
6.26k
•
515
tencent/SRPO
Updated
7 days ago
•
7k
•
869
openbmb/VoxCPM-0.5B
Updated
3 days ago
•
2.19k
•
481


In [22]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nAlibaba-NLP/Tongyi-DeepResearch-30B-A3B\nUpdated\n5 days ago\n•\n6.26k\n•\n516\ntencent/SRPO\nUpdated\n7 days ago\n•\n7k\n•\n869\nopenbmb/VoxCPM-0.5B\nUpdated\n3 days ago\n•\n2.19k\n•\n481\nibm-granite/granite-docling-258M\nUpdated\n4 days ago\n•\n16.4k\n•\n446\ngoogle/vaultgemma-1b\nUpdated\n10 days ago\n•\n3.25k\n•\n359\nBrowse 1M+ models\nSpaces\nRunning\n14k\n14k\nDeepSite v2\n🐳\nGener

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}


# Hugging Face Brochure

## About Us
**Hugging Face** is a pioneering community-driven platform focused on the development of artificial intelligence (AI) technologies, committed to building the future of machine learning (ML). As a hub for collaboration, Hugging Face empowers individuals and organizations to create, discover, and share innovative models, datasets, and applications.

## Our Offerings
- **Models**: Access over 1 million pre-trained ML models to kickstart your projects.
- **Datasets**: Browse and utilize over 250,000 datasets tailored for various ML tasks.
- **Spaces**: Create interactive applications effortlessly with our user-friendly interface.

## Industry Presence
More than **50,000 organizations**, including industry leaders such as Google, Amazon, Microsoft, and Intel, rely on Hugging Face’s solutions. Our technology serves a wide range of sectors, benefiting teams from non-profits to global enterprises.

## Community Engagement
### Join the Movement
Engage with a vibrant community of developers, researchers, and AI enthusiasts. Collaborate on projects, contribute to open-source tools, and stay updated with the latest advancements in machine learning.

- **Open Source Tools**: Participate in the development of essential tools like Transformers and Diffusers, which have become foundational for AI research. 
- **Community Contributions**: With over 150,000 contributions, our open-source projects are constantly updated and improved by our dedicated community.

## Company Culture
At **Hugging Face**, we cultivate a culture of inclusivity, innovation, and collaboration. Our team is composed of passionate individuals who are driven to expand the possibilities of AI. We believe in empowering our employees to pursue challenging projects and share their knowledge with the community. 

### Career Opportunities
Join us at Hugging Face! We are looking for talented individuals who are eager to make an impact in the field of AI and ML. We offer various roles across engineering, data science, and product management. By fostering a flexible work environment and providing opportunities for professional growth, we strive to create a workplace where everyone can thrive.

## Get In Touch
Whether you're a potential customer looking to leverage AI for your business, an investor seeking to support an innovative platform, or a recruit eager to join a forward-thinking team, Hugging Face invites you to connect with us.

- **Website**: [huggingface.co](https://huggingface.co)
- **Follow us on Social Media**: 
  - [Twitter](https://twitter.com/huggingface)
  - [LinkedIn](https://www.linkedin.com/company/huggingface)

Join us in shaping the future of AI!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [25]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)
    return response

In [26]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Welcome to Hugging Face: The AI Community with a Heart

---

### 🤗 The Company That Hugs You with AI

At Hugging Face, we believe that the power of Artificial Intelligence should not come with a user manual that’s thicker than a phone book! In fact, we’re on a mission to build the future through community collaboration, models, and maybe a few awkwardly enthusiastic high-fives along the way.  

### 🌟 What We Offer

- **1M+ Models & 250k+ Datasets**: It’s like a candy store for machine learning enthusiasts, but without the sticky floors (we promise!).
- **Spaces**: Collaborate and create AI applications faster than you can say “model-overload!” 
- **Enterprise Solutions**: For those high-flying companies that want AI without the fuss. 

### 🚀 Our Customers Are Out of This World!

Over 50,000 organizations (yes, you read that right) trust us with their AI needs, including industry giants like Amazon, Microsoft, and... well, we can’t forget about that little startup called Google. 

### 🤖 The Community That Keeps on Hugging!

Thanks to our vibrant community, you can tap into a treasure trove of knowledge and support. From enthusiastic learners to seasoned ML veterans, everyone here is ready to share, collaborate, and maybe sneer at your badly formatted code (but in a loving way).

### 🍕 The Company Culture: A Slice of Life at Hugging Face

Here at Hugging Face, we believe work should be as fun as a pizza party (without the heartburn). Our motto:
- **Collaborate, Don’t Compete**: Like friendly robots in a dance-off.
- **Open Source, Open Minds**: Because a closed source is like a closed box—who knows what’s in there? Might be old sandwiches!
- **Community First**: Our team is just as happy to say, “Let’s build something amazing” as they are to ask, “Who ordered extra toppings?”

### 👩‍💼 Careers: Join the Hugging Revolution!

Thinking about joining our team? Prepare for a journey where innovation meets community, and you can even contribute to a million models (that's right, we're not hoarding models like some AI dragon!).

- **Open Positions**: From enthusiastic interns to legendary engineers. Check out our job openings and let’s create AI magic together!
- **Perks of Working Here**: Tasty snacks, friendly colleagues, and the joy of knowing you're changing the future—how's that for some sweet job satisfaction?

---

### 📣 Final Thoughts

Whether you're a budding data scientist, an investor looking to leave your mark, or simply someone who believes in AI with a sprinkle of humor, Hugging Face is the place for you! Join us in our quest to build, collaborate, and occasionally hug.

---
**Ready to dive in? Sign up, browse our models, and let’s get hugging! 🤗**

"# Welcome to Hugging Face: The AI Community with a Heart\n\n---\n\n### 🤗 The Company That Hugs You with AI\n\nAt Hugging Face, we believe that the power of Artificial Intelligence should not come with a user manual that’s thicker than a phone book! In fact, we’re on a mission to build the future through community collaboration, models, and maybe a few awkwardly enthusiastic high-fives along the way.  \n\n### 🌟 What We Offer\n\n- **1M+ Models & 250k+ Datasets**: It’s like a candy store for machine learning enthusiasts, but without the sticky floors (we promise!).\n- **Spaces**: Collaborate and create AI applications faster than you can say “model-overload!” \n- **Enterprise Solutions**: For those high-flying companies that want AI without the fuss. \n\n### 🚀 Our Customers Are Out of This World!\n\nOver 50,000 organizations (yes, you read that right) trust us with their AI needs, including industry giants like Amazon, Microsoft, and... well, we can’t forget about that little startup cal

In [23]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Welcome to Hugging Face! 🤗

### The AI Community Building the Future (and maybe a solid breakfast too)

---

**🚀 What We Do:**  
At Hugging Face, we're all about collaboration and innovation in artificial intelligence! We provide a vibrant platform where the machine learning community gathers to create, discover, and share amazing models, datasets, and applications. Think of us as the social network for AI, where everyone gets to be a model citizen… literally!

**🌟 Highlights:**
- **1M+ Models** at your fingertips - Yes, you read that right! Choose from the most extensive collection of models, including those fancy language models everyone is raving about.
- **400k+ Apps** that will make you go, "Whoa! That's cool!" (And yes, some even make coffee!)
- **Datasets Galore**: Explore the wild world of datasets with over 250k at your disposal! You’ll never have to beg for data again. 

---

### 🤝 Join Our Fabulously Friendly Community
With over 50,000 organizations (including tech giants like **Google, Microsoft, and Amazon**), we’re a big family! If you ever feel lonely, just know that more than 30k people are *following* our models like they’re the latest trend in fashion!

---

### 💼 Careers with a Hug!
We get it; work can be a drag. That's why at Hugging Face, we have a culture that encourages collaboration, creativity, and the kind of fun that can only be had over a virtual cup of coffee. Join us for a career where:
- We *embrace* AI (pun intended), collaboration, and of course, a great work-life balance.
- Everyone has a say! We believe in sharing ideas (and memes) like they’re the latest viral TikTok dance.
- Expect to improve your ML skills faster than you can say "GPU." 

> **Are you ready to join a company where your ideas can literally *get trained*?**

---

### 💰 Investors' Corner
If you’re watching AI growth make your heart race, you’re not alone! Investing in Hugging Face means being part of an ecosystem that’s transforming the future (and making it **hugely** more fun!). 

**Why invest with us?**
- **A solid foundation** in open-source.
- **Exclusive enterprise solutions** to suit everyone from startups to previous rocket scientists at NASA.

---

### 🌯 In Conclusion
Whether you're a customer looking for cutting-edge AI tools, a potential recruit wanting to help shape the future, or an investor watching your money grow like a well-tended garden, Hugging Face is *the* place to be!

So why wait? Join the Hugging Face family today and let’s build the future together, one model at a time! 🤖

---

**Did someone say AI-powered hugs?** Because at Hugging Face, we don’t just sneak in awesome tech; we’re about spreading the love too! 💖

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>

Translate the brochure to other language - here English to Hindi

In [24]:
language_system_prompt = "You are an expert English to Hindi translator. You translate the English transcript without compromising the nuances of the language and the tone of the content."

In [27]:
def translate_brochure_English_to_Hindi(company_name, url):
    original_response = stream_brochure(company_name, url)
    translate_stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": language_system_prompt},
            {"role": "user", "content": original_response}
          ],
        stream=True
    )
    
    translated_response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in translate_stream:
        translated_response += chunk.choices[0].delta.content or ''
        translated_response = translated_response.replace("```","").replace("markdown", "")
        update_display(Markdown(translated_response), display_id=display_handle.display_id)
    

In [28]:
translate_brochure_English_to_Hindi("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Welcome to the Hugging Face Brochure!

## Where AI Dreams Come True (with a side of a friendly hug!) 🤗

**Company Overview:**  
At Hugging Face, we are more than just an AI community – we're a digital support group for machine learning enthusiasts! Whether you're looking to explore 1 million models (yes, that's the actual number!) or collaborate on groundbreaking datasets, we have it all. Ever seen a robot hug? Well, that’s us in spirit!

### Our Culture: 👐

- **Collaboration is Key:** We believe in teamwork… like that one time we tried to assemble IKEA furniture together but ended up just being great at sharing memes.
  
- **Open Source for the Win:** Our platform is like a buffet – only better! Grab what you like from our stack of open-source tools, but don’t forget to leave some for the others!

- **Diversity is Our Superpower:** Whether you're a seasoned data scientist or just debugging your first line of Python, we welcome everyone into our big family. (No training wheels needed!)

### Our Customers: 🌍

With over **50,000 organizations** (yep, that's a fifth of all organizations in the universe… we like to think), we serve folks from **AI2**, **Google**, and **Microsoft**, to the dog named **Fluffy** who has an admiration for conversational AI. Whether you’re an enterprise giant or a startup trying to build the next AI sensation, we’ve got your back.

### Career Opportunities: 💼

Want to join the Hugging Face family? Here’s what we look for in our recruits:

- **Passion for AI:** Must love AI more than your morning coffee (although coffee is also welcome… our office is always stocked!).
  
- **Collaborative Spirit:** Be ready to hug it out in brainstorming sessions. (Okay, virtual hugs are acceptable, too.)
  
- **A Sense of Humor:** Because AI and jokes go hand in hand. Ever heard the one about the neural network? It's a bit dense, but it learns from its mistakes!

### What We Offer: 💖

- **Unlimited Collaboration:** Host as many public models and datasets as your heart desires! (Warning: May cause excessive joy!)
  
- **Compute Solutions:** Start at just **$0.60/hour** for a GPU. A price so low, it can give back to your bank account a warm hug!

- **Enterprise Solutions:** Offering advanced security and support tailored just for organizations, because no one likes a hacker at their hug-fest.

### Join Us! 🚀

Ready to take a leap into the future? Whether you’re a developer thirsty for knowledge or an investor looking to ride the next AI wave, let’s connect and build (and hug) together! 

**Follow us on [GitHub](https://huggingface.co)**, [Twitter](https://twitter.com/huggingface), [LinkedIn](https://linkedin.com/company/hugging-face), or join our chat over on [Discord](https://discord.com/invite/huggingface) where the magic (and occasional shenanigans) happens.

---

At Hugging Face, tomorrow's AI is built today. And remember, we’re only a hug (or click) away!

# हगिंग फेस ब्रोशर में आपका स्वागत है!

## जहाँ एआई सपने सच होते हैं (एक दोस्ताना गले के साथ!) 🤗

**कंपनी का अवलोकन:**  
हगिंग फेस पर, हम सिर्फ एक एआई समुदाय नहीं हैं – हम मशीन लर्निंग उत्साही लोगों के लिए एक डिजिटल सपोर्ट ग्रुप हैं! चाहे आप 1 मिलियन मॉडल खोजने की कोशिश कर रहे हों (हाँ, यही असली संख्या है!) या ग्राउंडब्रेकिंग डेटा सेट पर सहयोग करने के लिए, हमारे पास सब कुछ है। क्या आपने कभी एक रोबोट को गले लगाते हुए देखा है? खैर, आत्मा में यही हैं हम!

### हमारी संस्कृति: 👐

- **सहयोग महत्वपूर्ण है:** हम टीमवर्क में विश्वास करते हैं… जैसे उस बार जब हमने साथ में IKEA का फर्नीचर लगाने की कोशिश की लेकिन अंततः मेम्स साझा करने में महान बन गए।

- **ओपन सोर्स का जश्न:** हमारा प्लेटफार्म एक बुफे की तरह है – सिर्फ बेहतर! हमारे ओपन-सोर्स टूल्स की ढेर से जितना चाहें उठाएं, लेकिन दूसरों के लिए कुछ छोड़ना न भूलें!

- **विविधता हमारी सुपरपावर है:** चाहे आप एक अनुभवी डेटा वैज्ञानिक हों या सिर्फ अपने पहले Python लाइन को डिबग कर रहे हों, हम अपने बड़े परिवार में सभी का स्वागत करते हैं। (कोई प्रशिक्षण पहिये की आवश्यकता नहीं!)

### हमारे ग्राहक: 🌍

**50,000 संगठनों** (हाँ, यही तो पूरे ब्रह्मांड के सभी संगठनों का पांचवां भाग है… हम यही सोचते हैं) के साथ, हम **AI2**, **Google**, और **Microsoft** से लेकर उस कुत्ते **Fluffy** तक लोगों की सेवा करते हैं जिसे संवादात्मक एआई से प्रेम है। चाहे आप एक बड़ी कंपनी हों या एक स्टार्टअप जो अगली एआई सनसनी बनाने की कोशिश कर रहा हो, हम आपके साथ हैं।

### करियर के अवसर: 💼

क्या आप हगिंग फेस परिवार में शामिल होना चाहते हैं? यहाँ वह चीजें हैं जो हम अपने उम्मीदवारों में ढूंढते हैं:

- **एआई के प्रति जुनून:** सुबह की कॉफी से ज्यादा एआई को प्यार करना चाहिए (हालांकि कॉफी भी स्वागत है… हमारा कार्यालय हमेशा स्टॉक में है!)।
  
- **सहयोगात्मक भावना:** ब्रेनस्टॉर्मिंग सत्रों में गले लगाने के लिए तैयार रहें। (ठीक है, वर्चुअल गले भी स्वीकार्य हैं।)
  
- **हास्य की भावना:** क्योंकि एआई और मजाक साथ-साथ चलते हैं। क्या आपने न्युरल नेटवर्क के बारे में सुना है? यह थोड़ा घना है, लेकिन अपने गलतियों से सीखता है!

### हम क्या प्रदान करते हैं: 💖

- **अनंत सहयोग:** जितने चाहें सार्वजनिक मॉडल और डेटा सेट होस्ट करें! (चेतावनी: अत्यधिक खुशी का कारण बन सकती है!)
  
- **कंप्यूट समाधान:** केवल **$0.60/घंटा** से GPU शुरू करें। एक कीमत इतनी कम, कि यह आपके बैंक खाते को एक गर्म गले में ढक सकती है!

- **एंटरप्राइज समाधान:** उन्नत सुरक्षा और समर्थन प्रदान करना, जो संगठनों के लिए विशेष रूप से अनुकूलित है, क्योंकि कोई भी अपनी गले लगाने की महफिल में हैकर नहीं चाहता।

### हमारे साथ जुड़ें! 🚀

क्या आप भविष्य में कूदने के लिए तैयार हैं? चाहे आप ज्ञान के प्यासे डेवलपर हों या अगले एआई लहर पर सवारी करने के लिए देख रहे निवेशक, चलिए जुड़ते हैं और मिलकर (और गले लगाते) हैं!

**हमारी सोशल मीडिया पर फ़ॉलो करें [GitHub](https://huggingface.co), [Twitter](https://twitter.com/huggingface), [LinkedIn](https://linkedin.com/company/hugging-face), या हमारे चैट में [Discord](https://discord.com/invite/huggingface) पर हमसे जुड़ें जहाँ जादू (और कभी-कभी शरारतें) होती हैं।**

---

हगिंग फेस पर, कल का एआई आज बनाया जाता है। और याद रखें, हम बस एक गले (या क्लिक) की दूरी पर हैं!